<a href="https://colab.research.google.com/github/HagiwaraNorito/DeepLearning2022Late/blob/main/%E5%8F%8E%E9%9B%86%E3%81%97%E3%81%9F%E7%94%BB%E5%83%8F%E3%81%A7%E5%AD%A6%E7%BF%92%E3%81%A8%E5%88%86%E9%A1%9E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#収集した画像をColabにUPして学習して、それをもとに分類させる

##ファイルのフォルダを準備する

In [26]:
import os
os.makedirs('sample',exist_ok=True)
os.makedirs('ourput/dog/sample',exist_ok=True)
os.makedirs('ourput/cat/sample',exist_ok=True)
os.makedirs('img/deep_leraning/dog',exist_ok=True)
os.makedirs('img/deep_leraning/cat',exist_ok=True)

##必要なライブラリをimport

In [27]:
import cv2
import numpy as np
import glob as glob
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

##分類（クラス）の名前をフォルダ名（dog,cat）から引用する

In [28]:
path = "img/deep_leraning"
folders=os.listdir(path)
classes=[f for f in folders if os.path.isdir(os.path.join(path,f))]
print(classes)
n_classes=len(classes)
print(n_classes)

['dog', 'cat']
2


##画像読み込みリサイズ、整形する

In [29]:
#画像とラベルのための配列
X=[]#画像
Y=[]#ラベル

#画像を読み込みつつリサイズする
for label,class_name in enumerate(classes):
  files = glob.glob(path + "/" + class_name + "/*jpg")
  for file in files:
    img=cv2.imread(file)
    img=cv2.resize(img,dsize=(224,224))
    X.append(img)
    Y.append(label)


##0～255のビットデータ値を、学習のために0～1に変換（正規化）学習精度を上げる

In [30]:
X=np.array(X)
X=X.astype('float32')
X /=255.0
#ラベルもnumpyの配列に変換
Y=np.array(Y)
Y=np_utils.to_categorical(Y,n_classes)

##整形したデータを学習用とテスト検証用に振り分ける

In [31]:
#学習データ8割、テスト検証用が2割に分ける
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2)
#学習データが8割
print(X_train.shape)
#テストでーた2割
print(X_test.shape)
#学習性データ8割（正解）
print(Y_train.shape)
#テストデータ2割(正解)
print(Y_test.shape)

(119, 224, 224, 3)
(30, 224, 224, 3)
(119, 2)
(30, 2)


#準備したデータを学習させていく

##学習に必要なライブラリをimport

In [32]:
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.models import Model
from keras.layers import Input,Activation,Dense,Flatten,Dropout
from keras.optimizers import Adam

##モデルのVGG16をアレンジしていく

In [33]:
#VGG16　入力層を用意する
input_tensor=Input(shape=(224,224,3))
#VGG16のインスタンスを作る(最後のsoftmaxの1000層を無効にして省く  )
base_model=VGG16(weights='imagenet',input_tensor=input_tensor,include_top=False)


In [34]:
top_model=Sequential()
top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
top_model.add(Dense(n_classes,activation='softmax'))

##最後のSOFTMAXを外したモデル（base_model）と追加用の2つに分類するSOFTMAX層のモデルに（top_model）つなげる

In [35]:
model=Model(inputs=base_model.input,outputs=top_model(base_model.output))

In [36]:
for layer in model.layers[:15]:
  layer.trainable=False
print('# laers=',len(model.layers))

# laers= 20


In [37]:
#損失はクロスエントロピー法で算出、最適化はADAM、指標は精度で見る
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

###コンパイルしたモデルを表示

In [38]:
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

##コンパイルしたモデルで収集した画像データを学習させる

In [39]:
model.fit(X_train,Y_train,epochs=20,batch_size=16)

Epoch 1/20
8/8 [==============================] - 1s 91ms/step - loss: 1.2899 - accuracy: 0.4454
Epoch 2/20
8/8 [==============================] - 1s 85ms/step - loss: 0.9808 - accuracy: 0.5042
Epoch 3/20
8/8 [==============================] - 1s 85ms/step - loss: 0.7437 - accuracy: 0.4874
Epoch 4/20
8/8 [==============================] - 1s 85ms/step - loss: 0.7346 - accuracy: 0.4538
Epoch 5/20
8/8 [==============================] - 1s 85ms/step - loss: 0.6988 - accuracy: 0.5546
Epoch 6/20
8/8 [==============================] - 1s 86ms/step - loss: 0.9885 - accuracy: 0.5630
Epoch 7/20
8/8 [==============================] - 1s 85ms/step - loss: 0.6886 - accuracy: 0.5966
Epoch 8/20
8/8 [==============================] - 1s 85ms/step - loss: 0.6487 - accuracy: 0.6050
Epoch 9/20
8/8 [==============================] - 1s 85ms/step - loss: 0.6146 - accuracy: 0.6555
Epoch 10/20
8/8 [==============================] - 1s 85ms/step - loss: 0.5979 - accuracy: 0.7059
Epoch 11/20
8/8 [============

##学習の成果を指標(accuracy)で確認

In [40]:
score=model.evaluate(X_test,Y_test,batch_size=16)

2/2 [==============================] - 0s 92ms/step - loss: 1.7085 - accuracy: 0.5667


##作成した学習モデルとクラス分類名を保存

In [41]:
import pickle
pickle.dump(classes,open('classes.sav','wb'))
model.save('cnn.h5')

#保存したクラス分類と学習モデルを読み込んで使う

In [42]:
#ライブラリのimport
from keras.models import load_model
import pickle
import cv2
import glob

##ファイルを読み込んでオブジェクトとしてよみがえらせる

In [43]:
model=load_model('cnn.h5')
classes=pickle.load(open('classes.sav','rb'))

##sample画像を読み込む

In [46]:
files=glob.glob('sample/*')
print(files)

['sample/cat_79.jpg', 'sample/dog_11.jpg', 'sample/dog_14.jpg']


##ファイルパスから画像を読み込み、学習モデルの判定させ、該当するoutputフォルダにコピーを書き込む

In [49]:
for file in files:
  img=cv2.imread(file)
  img2=cv2.resize(img,dsize=(224,224))
  img2=img2.astype('float32')
  img2=img2/255.0
  img2=img2[None,...]
  result=model.predict(img2)
  print(result)
  pred=result.argmax()
  print(pred,str(classes[pred]))
  cv2.imwrite('output/'+str(classes[pred])+'/'+file,img)


1/1 [==============================] - 0s 26ms/step
[[2.1413295e-07 9.9999976e-01]]
1 cat
1/1 [==============================] - 0s 15ms/step
[[1.000000e+00 4.751215e-09]]
0 dog
1/1 [==============================] - 0s 17ms/step
[[9.9995816e-01 4.1817919e-05]]
0 dog
